In [23]:
####
# @author: Nwamaka Nzeocha and Fabian Okeke
# @course: CS 5304/Data Science in the Wild
####

# Environment setup

 - useful tensor flow notebook: http://bit.ly/1NjhcfQ
 - pip install scikit-learn
 - pip install --upgrade https://storage.googleapis.com/tensorflow/mac/tensorflow-0.8.0-py2-none-any.whl
 - dataset:
     - https://s3.amazonaws.com/giyengar-cs5304/a4_smvl_tst.gz
     - https://s3.amazonaws.com/giyengar-cs5304/a4_smvl_val.gz
     - https://s3.amazonaws.com/giyengar-cs5304/a4_smvl_trn.gz

In [1]:
import random
import os.path
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from sklearn.linear_model import SGDRegressor


In [21]:
def exists(filepath):
    return os.path.exists(filepath)

def read_file(filepath):
    X, y = [], []
    with open(filepath) as f:
        for line in f:
            row = line.split()
            y.append(row[0])
            X.append(row[1:])
    
    
    return X,y

In [ ]:
X_train, y_train = read_file('a4_smvl_trn')

In [ ]:
X_train[:3]

In [ ]:
y_train[:3]

In [32]:
count = 0
X, y = [], []

limit = 5000
with open('train500') as f:
    for line in f:
        row = line.split()
        y.append(row[0])
        X.append(len(row))
        count += 1
        
        if count == limit: break
print len(y)

500


In [18]:
max(X)

416

In [12]:
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

@mem.cache
def get_data(filepath):
    data = load_svmlight_file(filepath)
    return data[0], data[1]

# Load train data

In [13]:
X_train, y_train = get_data('train500')

________________________________________________________________________________
[Memory] Calling __main__--Users-fnokeke-dev-cs5304-deepLearning-__ipython-input__.get_data-alias...
get_data-alias('train500')
___________________________________________________get_data-alias - 0.0s, 0.0min


In [30]:
X_train

<500x18859 sparse matrix of type '<type 'numpy.float64'>'
	with 14368 stored elements in Compressed Sparse Row format>

In [29]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [28]:
type(y_train)

numpy.ndarray

# Get dataset

In [ ]:
X_train, y_train = '', ''
X_va, y_va = '', ''

# Fit logistic regression model

In [ ]:
model_LR = LogisticRegression()
model_LR.fit(X_train, y_train)
predicted_LR = model_LR.predict(X_va)

model_LR

# Fit multi-layer perceptron model

In [ ]:
model_MLP = MultiLayerPerceptron()
model_MLP.fit(X_train, y_train)
predicted_MLP = model_MLP.predict(X_va)

model_MLP

# Pre calibration Brier loss and Brier curves

In [ ]:
print "pre-calibration brier loss:"
print "pre-calibration brier curve:"

# Post calibration Brier loss and Brier curves

In [ ]:
print "post-calibration brier loss:"
print "post-calibration brier curve:"

# ROC curves, Accuracy scores, AUC scores

In [ ]:
y_predictions = {
    'Logistic Regression': predicted_LR,
    'Tensor Flow': predicted_MLP,
}

fig, axes = plt.subplots(1, len(y_predictions), sharey=True)
fig.set_size_inches(15,5)
i = 0

for title, prediction in y_predictions.items():
    accuracy = metrics.accuracy_score(y_va, prediction)
    fulltitle = title + ' (accuracy = %0.2f)' % accuracy
    fpr, tpr, thresholds = metrics.roc_curve(y_va, prediction, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    label = 'AUC = %0.2f' % auc

    axes[i].plot(fpr, tpr, label=label)
    axes[i].set_title(fulltitle)
    axes[i].legend(loc='lower right')
    i += 1